In [1]:
import sys

sys.path.append("D:/Projekte/Sim/src")
import sim

In [2]:
cycle_time = 65

geo1 = sim.DoubleGeo("Geo1", ["Alle"], "Anschluss li", "Anschluss re", 20)
sim.BufferSource("Source Geo1", cycle_time, 0, parent = geo1, getNextPart=sim.src_part("Alle"))
geo2 = sim.DoubleGeo("Geo2", ["Alle"], "Ltr li", "Ltr re", 20)
sim.BufferSource("Source Geo2", cycle_time, 0, parent = geo2, getNextPart=sim.src_part("Alle"))

geo3l = sim.Geo("Geo3 li", ["Ltr li", "Anschluss li", "Sonst li"], "", 20)
sim.BufferDrain("Drain Geo3 li", cycle_time,  parent = geo3l)
geo3r = sim.Geo("Geo3 re", ["Ltr re", "Anschluss re", "Sonst re"], "", 20)
sim.BufferDrain("Drain Geo3 re", cycle_time,  parent = geo3r)

in_l= sim.Jig("In li")
sim.BufferSource("Source In li", 5, 0, parent = in_l, getNextPart=sim.src_part("Sonst li"))
in_r= sim.Jig("In re")
sim.BufferSource("Source In re", 5, 0, parent = in_r, getNextPart=sim.src_part("Sonst re"))


In [3]:
z_g3r_g2 = sim.Zone("Achse Bereich g3r/g2")
z_g3l_g1 = sim.Zone("Achse Bereich g3l/g1")

Rob1 = sim.Rob("Roboter 1")
sim.RobHandlingBetter("1->3r",[
    sim.Pick(geo1, 6, certain_part="Anschluss re", start_condition=sim.cond_buffer_willing_receive(geo3r, "Anschluss re"), blocking = z_g3l_g1), 
    sim.Place(geo3r, 6, 5, blocking=z_g3r_g2)
], parent = Rob1)
sim.RobHandlingBetter("2->3r",[
    sim.Pick(geo2, 6, certain_part="Ltr re" , start_condition=sim.cond_buffer_willing_receive(geo3r, "Ltr re"), blocking = z_g3r_g2), 
    sim.Place(geo3r, 6, 5, blocking = z_g3r_g2)
], parent = Rob1)
sim.RobHandlingBetter("1->3l",[
    sim.Pick(geo1, 6, certain_part="Anschluss li" , start_condition=sim.cond_buffer_willing_receive(geo3l, "Anschluss li"), blocking = z_g3l_g1), 
    sim.Place(geo3l, 6, 5, blocking = z_g3l_g1)
], parent = Rob1)
sim.RobHandlingBetter("2->3l",[
    sim.Pick(geo2, 6, certain_part="Ltr li" , start_condition=sim.cond_buffer_willing_receive(geo3l, "Ltr li"), blocking = z_g3r_g2), 
    sim.Place(geo3l, 6, 5, blocking = z_g3l_g1)
], parent = Rob1)
Rob2 = sim.Rob("Roboter 2")
sim.RobHandlingBetter("in_l->3l",[
    sim.Pick(in_l, 6, start_condition=sim.cond_buffer_willing_receive(geo3l,"Sonst li")),
    sim.Place(geo3l, 6, 5, blocking = z_g3l_g1)
], parent = Rob2)
sim.RobHandlingBetter("in_r->3r",[
    sim.Pick(in_r, 6, start_condition=sim.cond_buffer_willing_receive(geo3r, "Sonst re")),
    sim.Place(geo3r, 6, 5, blocking=z_g3r_g2)
], parent = Rob2)

In [4]:
sim.core.run(0, 60 * 60)

In [5]:
sim.core.plotHistory(excludes = ["Fill"])